# Logistics and operations planning with Médecins Sans Frontières - Workshop data
### Prepared by Kit Daniel Searle and Andries Heyns
This notebook is to load the data containing information pertaining to dwellings and candidate medical facilities. We also provide some functions for some data processing and generating visualisations.

The first thing we will do is import all necessary packages and modules.

In [ ]:
from class_dwellings import Dwellings
from class_candidates import Candidates
from obj_func_code import process_raster
from plots import plot_result_with_map, plot_dwellings, plot_candidates, plot_candidate_raster
from pathlib import Path
from zipfile import ZipFile
import py7zr

## Initialize Data Folders
This section sets up the working environment. It checks whether the required data folders (`v_b`, `v_i`, `v_t`) are already extracted. If not, it automatically extracts them from corresponding `.zip` and `.7z` files. It also ensures that temporary and plotting output directories exist.

You only need to ever run this code once **and this may take a while to run**.

In [ ]:
cwd = Path.cwd()  # or your specific project path
data_folder = cwd / "data" 

temp_data_folder = cwd / "data" / "temp_data"
plots_folder = cwd / "plots" 

folder_names = ["v_b", "v_i", "v_t"]

for name in folder_names:
    folder = data_folder / name
    zip_file = data_folder / f"{name}.zip"
    z7_file = data_folder / f"{name}.7z"

    if folder.exists():
        print(f"✅ Folder exists: {folder}")
        continue

    if zip_file.exists():
        print(f"📦 Unzipping: {zip_file.name} …")
        with ZipFile(zip_file, "r") as z:
            z.extractall(data_folder)
        print(f"✅ Unzipped → {folder}")
    elif z7_file.exists():
        print(f"📦 Unzipping: {z7_file.name} …")
        with py7zr.SevenZipFile(z7_file, mode='r') as archive:
            archive.extractall(path=data_folder)
        print(f"✅ Unzipped → {folder}")
        
    else:
        print(f"❌ Neither folder nor zip found for: {name}")

# Check if it exists, create if not
temp_data_folder.mkdir(parents=True, exist_ok=True)
plots_folder.mkdir(parents=True, exist_ok=True)

## Dwellings class
This step loads household dwelling information and associated raster layers. 

The `Dwellings` class is designed to load and store data from two raster files related to dwelling counts and dwelling isolation. It provides convenient access to both raster values and spatial metadata. Some important atribtes include:

- **main_transform**: Geo-transform mapping pixel to coordinate space.
- **main_width / main_height**: Raster dimensions.
- **main_data**: Array of dwelling counts.
- **isolation_data**: Array of normalized isolation values. This array takes a value of 1 if a dwelling is isolated and 0 if there are many dwellings in the surroundings.

For example `dwellings.main_data` returns a numpy array with dimensions `main_width` by `main_height`. If we want to get the coordinates  of a specific cell in the array we can use the function `dwellings.get_cordinates(r,c)` where `r` is the row and `c` is the column.


In [ ]:
# Create an instance of the Dwellings class, pointing to the folder containing the raster files
dwellings = Dwellings(data_folder)

# Load raster data and metadata (reads the TIFF files into memory)
dwellings.load_data()

# Specify a row and column index from the main raster array
row = 944
column = 1149

# Convert the row/column index to map (x, y) coordinates using the raster transform
coordinates = dwellings.get_coordinates(row, column)
print(f'Coordinates at {(row,column)} are {coordinates}')

# Get the number of dwellings at this location
number_of_dwellings = dwellings.main_data[row,column]
print(f'Number of dwellings at {(row,column)} are {number_of_dwellings}')

# Get the number of dwellings at this location
isolated_value = dwellings.isolation_data[row,column]
print(f'The dwellings at {(row,column)} have an isolation value of {isolated_value}')

Finally, we can plot all the dwellings.

In [ ]:
plot_dwellings(dwellings)

## **Candidates class**

The `Candidates` class manages a set of candidate site locations. It loads candidate points from a GeoJSON file, builds spatial relationships between them, and links each site to raster-based attributes used in analysis and optimization.

---

####  **Loading Candidate Coordinates**
The first thing we will look at is the coordinates of the candidate locations. The `Candidates` class has a method called `load_candidate_locations()` which reads the GeoJSON file and extracts each site’s:
- **ID**
- **Coordinate pair (longitude, latitude)**  

This is done in the constructor so all we need to do is create an object called `candidates`. If we want to get the location of a candidate location we can just reference the dictionary `candidates.all_candidates`. For example


In [ ]:
# Create a Candidates object and load candidate location data
candidates = Candidates(data_folder)

# Select the candidate ID we want to inspect
candidate_number = 3998

# Retrieve the coordinate pair (x, y) for that candidate
location = candidates.all_candidates[candidate_number]

# Print the candidate ID and its coordinates
print(f'Candidate {candidate_number} is located at {location}')

Next, we can plot all of the candidate locations on a map.

In [ ]:
plot_candidates(candidates)



Next, we want to load all the data related to the candidate locations. The `load_data()` method loads raster datasets associated with each candidate site, aligns them to a common reference raster, cleans their values, and stores the results for later use in analysis or optimization.

---

#### **What This Method Does**

1. **Iterates over each candidate site**
   - Each candidate has its own set of raster layers stored in:
     - `v_i/` → reduction layer  
     - `v_b/` → coverage layer  
     - `v_t/` → travel time layer  

2. **Stores processed raster data**
   - For each candidate, the method saves:
     ```
     {
       'window': <alignment window on main raster>,
       'reduction': <reduction raster values>,
       'coverage': <coverage raster values>,
       'time': <travel time raster values>
     }
     ```
   - This is stored in:
     ```
     self.candidate_data[candidate]
     ```
Lets see an example of this. 

**Note, loading the data may take a while.**

In [ ]:
# Load all candidate raster data and align them to the main raster
candidates.load_data()

# Choose a specific candidate ID to inspect
candidate_to_check = 3998

# Get the spatial window (position) of this candidate's raster in the main raster
window = candidates.candidate_data[candidate_to_check]['window']
print(f'This candidate’s raster is positioned at {window} in the main data raster')

# Retrieve the candidate's processed reduction raster
reduction_raster = candidates.candidate_data[candidate_to_check]['reduction']

# Retrieve the candidate's processed coverage raster
coverage_raster = candidates.candidate_data[candidate_to_check]['coverage']

# Retrieve the candidate's processed travel time raster
travel_time_raster = candidates.candidate_data[candidate_to_check]['time']

# Print the travel time raster array
print(travel_time_raster)

We can visualise the rasters with the following function.

In [ ]:
plot_candidate_raster(candidates, candidate_to_check, 'reduction') # you can change 'time' to `coverage` or `reduction`

## I have added some helper functions for you

This computes spatial service influence regions (Voronoi polygons) around candidate sites and builds adjacency relationships used in coverage and pathing. They are:

`get_voronoi()`:
- Computes a **Voronoi diagram** from candidate coordinates.
- Uses shared Voronoi boundaries to create a **NetworkX graph**, where:
- Nodes = candidate sites  
- Edges = pairs of sites whose Voronoi cells touch  
This graph represents spatial adjacency between sites.

`get_nearest_neighbours(location, dist)`:
- Performs a graph-based search starting at a given candidate.
- Keeps neighbors within a specified **distance threshold**.
- Returns: neigbouring candidate locations

In [ ]:
# Construct the Voronoi adjacency graph for all candidates
# Each candidate becomes a node, and edges connect candidates whose Voronoi cells share a border
candidates.get_voronoi()

# Visualize the candidate network on a map
# Nodes = candidates, edges = neighbor relationships
candidates.plot_graph()

# Choose a specific candidate ID to inspect
candidate_to_check = 3998

# Find neighboring candidates within a scaled distance threshold (2x the minimum distance)
neighbours = candidates.get_nearest_neighbours(candidate_to_check, 2)

# Print the IDs of candidates that are considered neighbors
print(f'Candidates near to {candidate_to_check} are {list(neighbours.keys())}')


## Process Build Scenario and Plot Results

Finally, we put this all together by calling the function process_raster. The process raster function returns three components described in the table below.

| Return Value | Description | Interpretation |
|--------------|-------------|----------------|
| `total_sum_reduction` | Sum of reduction benefit weighted by population | How much total benefit the selected sites provide |
| `total_sum_coverage` | Sum of coverage benefit weighted by population | How many people are served by the selected sites |
| `total_sum_fairness` | Sum of coverage benefit weighted by area isolation | How well remote or underserved communities are supported |

In [ ]:
build = [3998, 28167]
result = process_raster(candidates, dwellings, build)
print(f'total reduction = {result[0]}, number of dwellings covered = {result[1]}, total equity = {result[2]}')
plot_result_with_map(build, candidates, dwellings, 'test')